# Análise das tendências políticas no município do Rio de Janeiro
## Por Omar Mesquita

Código do município do Rio de Janeiro: **3304557** (obtido no [site do IBGE](https://www.ibge.gov.br/cidades-e-estados/rj/rio-de-janeiro.html))

[Aqui](https://www.tse.jus.br/servicos-eleitorais/cartorios-e-zonas-eleitorais/pesquisa-a-zonas-eleitorais) pode-se ver as zonas eleitorais no **estado** do Rio de Janeiro. Por uma consulta
SQL podemos visualizar apenas zonas no **município** do Rio: <br><br>

```SQL

    SELECT ano, turno, sigla_uf, id_municipio, zona, numero_partido, resultado, votos
    FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio_zona`
    WHERE cargo='presidente'
      AND ano >= 2008
      AND sigla_uf='RJ'
      AND id_municipio = '3304557'
    LIMIT 100
```


In [1]:
# Por favor, execute essa célula antes de continuar
# a leitura para garantir as importações de módulos

import pandas as pd     # Leitura e tratamento de tabelas
import basedosdados as bd   # API para acessar dados do TSE
import os                   # Caminhos e I/O
import matplotlib.pyplot as plt # Gráficos

# Determina o diretório no qual o caderno está sendo executado
currentDir = os.getcwd()

Devido à elevada quantidade de zonas eleitorais no município do Rio de Janeiro, faz-se necessário escolher aquelas que contenham a maior quantidade de pessoas votantes. Essa atitude viabiliza a análise dos dados e maximiza o espaço amostral do estudo *(e?)*

Podemos obter essa informação na página oficial do Tribunal Superior Eleitoral de [estatísticas do eleitorado](https://www.tse.jus.br/eleicoes/estatisticas/estatisticas-de-eleitorado/consulta-por-municipio-zona).

Para esse estudo, foi baixada uma tabela em formato `.csv` contendo dados da última eleição (**outubro de 2022**) no **estado do Rio de Janeiro (RJ)**

**Obs.:** A tabela foi baixada e, em seguida, salva em formato **UTF-8** para evitar problemas de codificação.

In [16]:
# Armazena o arquivo .csv em um DataFrame
df = pd.read_csv(os.path.join(currentDir,"csv","quantidade_de_eleitores.csv"))

# Cria um DataFrame apenas com as colunas que serão utilizadas (Zona e Quantidade)
# e satisfazem a condição de estarem no município do Rio de Janeiro.
# Em seguida, ordena o DataFrame em ordem decrescente de quantidade de eleitores
# e seleciona as 4 zonas com maior quantidade de eleitores.
df_rj = df[ df["Município"] == "RIO DE JANEIRO"] [['Zona', 'Quantidade']].sort_values(by='Quantidade', ascending=False).head(4)

print(df_rj)

     Zona  Quantidade
113   119      148555
102     9      146008
124   179      140924
133   214      136384


A partir da visualização da tabela, nota-se que as zonas eleitorais **119**, **9**, **179** e **214** são as que concentram maior quantidade de pessoas votantes.

Dessa forma, é possível, agora, fazer uma análise dos resultados eleitorais dessas zonas.

Para tanto, baixou-se uma tabela com informações sobre os resultados das eleições presidenciais de 2022 nas zonas eleitorais previamente citadas. Foi utilizada a API Python do [Base dos Dados](https://basedosdados.org/) e uma consulta SQL no interior da função para se obter tais dados.

In [5]:
# AVISO! Caso você tenha baixado esse caderno ou o repositório,
# não é necessário executar essa célula, pois os dados já foram baixados
zonas = [119, 9, 179, 214]

for i in zonas:

    bd.download(
        savepath=os.path.join(currentDir,"csv",f"resultados_eleicoes_{i}.csv"),
        table_id="resultados_candidato_municipio_zona",
        billing_project_id="ufrjanalytica",
        query= f'''
        SELECT turno, zona, numero_partido, resultado, votos, sigla_partido
        FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio_zona`
        WHERE ano = 2022
          AND sigla_uf = "RJ"
          AND id_municipio = "3304557"
          AND cargo = "presidente"
          AND zona = {i}
        LIMIT 1000
        '''

)